In [2]:
import re
import pandas as pd
import numpy
import numpy.linalg as la
import datetime
from sklearn.preprocessing import LabelEncoder      # LabelEncoder

 
df_test_source = pd.DataFrame()
df_train_source = pd.DataFrame()    
    
print("Start time:{}".format(datetime.datetime.now()))

fp = open("punctuation.txt", "r", encoding = 'utf8')
raw_data = fp.read()
punctuation_data=[]
punctuation_data = raw_data.split("\n")
fp.close()

fp = open("stopword_chinese.txt", "r", encoding = 'utf8')
raw_data = fp.read()
stopword_chinese_data=[]
stopword_chinese_data = raw_data.split("\n")
fp.close()

item_id_train = []
class_label_train = []
article_train =[]

fp = open("TrainingData.txt", "r", encoding = 'utf8')
raw_data = fp.readlines()
for item in raw_data :
    item_id_train.append(item.split("\t")[0])
    class_label_train.append(item.split("\t")[1])  
    text = item.split("\t")[2].strip("\n") 
    new_text = ''.join([c for c in text if c not in punctuation_data])
   
    article_train.append(new_text)
    
fp.close()

df_train_source["id"] = item_id_train
df_train_source["class_label"] = class_label_train
df_train_source["article"] = article_train


item_id_test = []
article_test =[]

fp = open("TestData.txt", "r", encoding = 'utf8')
raw_data = fp.readlines()
for item in raw_data :
    item_id_test.append(item.split("\t")[0]) 
    text = item.split("\t")[1].strip("\n") 
    new_text = ''.join([c for c in text if c not in punctuation_data])
    article_test.append(new_text)
       
fp.close()

df_test_source["id"] = item_id_test
df_test_source["article"] =article_test




Start time:2017-05-24 00:04:18.859034


In [3]:
import numpy as np

class_mapping = {label:idx for idx,label in enumerate(np.unique(df_train_source["class_label"]))}
#class_mapping

df_train_source["class_label"] = df_train_source["class_label"].map(class_mapping)


In [4]:


X_train = df_train_source.loc[:,'article'];
y_train = df_train_source.loc[:, 'class_label'];
X_test =  df_test_source.loc[:,'article'];


In [5]:
# 文件轉為字符
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):  # 方法1. 空白字元斷字
    return text.split()
def tokenizer_porter(text): # 方法2. 字詞 -> 字根
    return [porter.stem(word) for word in text.split()]

In [6]:
import sys
import os
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB


    
# Create feature vectors
vectorizer = TfidfVectorizer(min_df=5,
                             max_df = 0.8,
                             sublinear_tf=True,
                             stop_words= stopword_chinese_data,
                             #tokenizer=tokenize,
                             use_idf=True)
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)



classifier_MultinomialNB = MultinomialNB(alpha=1.0, fit_prior=True)
fit_MultinomialNB = classifier_MultinomialNB.fit(train_vectors, y_train)
prediction_MultinomialNB = classifier_MultinomialNB.predict(test_vectors)




In [7]:
df_test_source["predict_MultinomialNB"] = prediction_MultinomialNB

In [8]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df_test_source["Similarity_MultinomialNB"] = df_test_source["predict_MultinomialNB"].map(inv_class_mapping)

In [9]:
df_test_source.head(50)

,id,article,predict_MultinomialNB,Similarity_MultinomialNB
0,3501,觀念 平台 當 林書豪 遇到 大學 學測 中國時報 周祝瑛 不久 前 當 哈佛 小子 ...,1,sports
1,3502,觀光局 自由行 試點 城市 磋商 中 中央社 記者 謝佳珍 台北 23日 電 外傳 陸...,0,politics
2,3503,讓 愛 傳出去 周美青 歲末 感恩 志工 選戰 進入 倒數 計時 總統 夫人 酷酷 嫂 ...,0,politics
3,3504,驕傲 公雞 在 阻撓 國 親 合 該 先 清君側 備受 關注 的 國民黨 不 分區 立委 ...,0,politics
4,3505,魔術 強森 讚 林書豪 讓 球隊 更 好 中央社 記者 吳協昌 洛杉磯 7日 專電 父母...,1,sports
5,3506,魔術 強森 林 像 奈許 與 史托克頓 中央社 記者 吳協昌 洛杉磯 10日 專電 父...,1,sports
6,3507,鐵馬 瘋子 鄭文章 肺 腺癌 過世 中央社 記者 李宇政 台北 10日 電 台灣 超級 ...,1,sports
7,3508,蘭嶼 怒吼 要求 遷 核 廢料 原能會 踢 皮球 給 台電 經濟部 政治 中心 綜合 報導...,0,politics
8,3509,黨魁 之 爭 民進黨 有效票 16萬 備受 矚目 的 民主進步黨 主席 選舉 即將 起跑 ...,0,politics
9,3510,黨 內 五 大老 同 台 吳伯雄 並 不 困難 國民黨 將 在 十月 十五號 成立 第一 ...,0,politics
